In [1]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
DATA_DIR = os.path.join('..', 'data')

In [5]:
train_cust =pd.read_csv(os.path.join(DATA_DIR, 'train_customers.csv'), parse_dates=['created_at','updated_at'])
test_cust =pd.read_csv(os.path.join(DATA_DIR, 'test_customers.csv'), parse_dates=['created_at','updated_at'])
train_loc =pd.read_csv(os.path.join(DATA_DIR, 'train_locations.csv'))
test_loc =pd.read_csv(os.path.join(DATA_DIR, 'test_locations.csv'))
vendors =pd.read_csv(os.path.join(DATA_DIR, 'vendors.csv'), parse_dates=['created_at','updated_at'])
orders =pd.read_csv(os.path.join(DATA_DIR, 'orders.csv'), parse_dates=['created_at','delivery_date'])
submission =pd.read_csv(os.path.join(DATA_DIR, 'SampleSubmission.csv'))

In [6]:
train_cust= train_cust.rename({'akeed_customer_id': 'customer_id'}, axis=1)
test_cust= test_cust.rename({'akeed_customer_id': 'customer_id'}, axis=1)

In [7]:
test_cust=test_cust[['customer_id','created_at','updated_at']]
train_cust=train_cust[['customer_id','created_at','updated_at']];train_cust.head()

,customer_id,created_at,updated_at
0,TCHWPBT,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,2018-03-15 19:57:01,2018-03-15 19:57:01


In [8]:
test_cust = test_cust.drop_duplicates(subset='customer_id', keep="first")
train_cust = train_cust.drop_duplicates(subset='customer_id', keep="first")

In [9]:
train =pd.merge(train_cust, train_loc, on ='customer_id', how ='left')
test =pd.merge(test_cust, test_loc, on ='customer_id', how ='left')

In [10]:
test=test.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)
train=train.assign(key=1).merge(vendors.assign(key=1), on='key').drop('key',axis=1)

In [11]:
test['location_number_obj']=test['location_number'].astype(str)
test['id_obj']=test['id'].astype(str)
train['location_number_obj']=train['location_number'].astype(str)
train['id_obj']=train['id'].astype(str)

In [12]:
test['CID X LOC_NUM X VENDOR'] = test['customer_id'] +' X '+ test['location_number_obj'] +' X '+ test['id_obj']
train['CID X LOC_NUM X VENDOR'] = train['customer_id'] +' X '+ train['location_number_obj'] +' X '+ train['id_obj']

In [13]:
test = pd.merge(test, submission, how='outer', on=['CID X LOC_NUM X VENDOR'])

In [14]:
all_services = orders['CID X LOC_NUM X VENDOR'].values.tolist()
train['target']= train['CID X LOC_NUM X VENDOR'].isin(all_services).astype(int).values

In [15]:
train['target'].value_counts()

0    5724146
1      78254
Name: target, dtype: int64

In [16]:
test.drop('target', axis=1, inplace =True)

In [18]:
print(test.shape, train.shape)

(1672000, 69) (5802400, 70)


In [19]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [20]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 2318.58 Mb (25.7% reduction)
Mem. usage decreased to 676.09 Mb (24.3% reduction)


In [21]:
train.to_csv(os.path.join(DATA_DIR, 'train.csv'), index =False)
test.to_csv(os.path.join(DATA_DIR, 'test.csv'), index =False)